<a href="https://colab.research.google.com/github/Stefano-t/bioinf-lab/blob/main/prediction_ae_hg38.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
print("installing dependency...")
!wget "https://raw.githubusercontent.com/LucaCappelletti94/bioinformatics_practice/master/requirements.txt" &> /dev/null
%pip install -r requirements.txt &> /dev/null
#print("downloading epigenomic_dataset...")
#%pip install epigenomic_dataset &> /dev/null && echo "epigenomic_dataset installed!"
print("downloading humanize 3.6.0...")
%pip install -Iv humanize==3.6.0 &> /dev/null && echo "humanize 3.6.0 installed!"
print("downloading extra_keras_metrics...")
%pip install extra_keras_metrics &> /dev/null && echo "extra_keras_metrics installed!"

!rm requirements.txt
print("finish!")

installing dependency...
scikit-learn
barplots
keras_history
aset>=1.1.7
2
quence>=1.1.5
ray
seaborn
nepy
boruta
ince
cmake
ticoretsne
w>=1.1.1
0:00:14 [================================================================>] 100%
downloading humanize 3.6.0...
humanize 3.6.0 installed!
downloading extra_keras_metrics...
extra_keras_metrics installed!
finish!


In [68]:
%tensorflow_version 2.x
import tensorflow as tf
from epigenomic_dataset import load_epigenomes
from sklearn.impute import KNNImputer
import pandas as pd
from sklearn.preprocessing import RobustScaler
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from tqdm.auto import tqdm
from tqdm.keras import TqdmCallback

from sklearn.tree import DecisionTreeClassifier
from cache_decorator import Cache

from typing import Dict, List
from sanitize_ml_labels import sanitize_ml_labels
from sklearn.metrics import accuracy_score, average_precision_score, roc_auc_score


In [15]:
# load the dataset

X, y = load_epigenomes(
    cell_line = "K562",
    dataset = "fantom",
    region = "enhancers",
    window_size = 256,
    root = "datasets"
)

In [16]:
# the strand in the enhancer is not relevant
# the columns have values which is equal to the average times the protein 
# apperers in the given window_size 
X

chrom                              SMAD5  NCOA2  ZNF23  ...  NCOR1  NCOA1  KLF1
chrom chromStart chromEnd  strand                       ...                    
chr10 100006381  100006637 .        0.90   0.90   1.37  ...   2.38   0.82  0.69
      100008146  100008402 .        0.73   1.06   0.74  ...   3.11   0.49  1.05
      100014418  100014674 .        0.77   0.33   0.53  ...   0.61   0.97  0.96
      100020216  100020472 .        0.87   0.80   0.83  ...   0.57   0.03  1.07
      100043528  100043784 .        0.64   0.53   0.03  ...   1.26   1.73  0.33
...                                  ...    ...    ...  ...    ...    ...   ...
chrY  7520247    7520503   .        0.00   0.00   0.00  ...   0.00   0.00  0.00
      7724272    7724528   .        0.00   0.00   0.00  ...   0.00   0.00  0.00
      7770029    7770285   .        0.00   0.00   0.00  ...   0.00   0.00  0.00
      7796295    7796551   .        0.00   0.00   0.00  ...   0.00   0.00  0.00
      8007562    8007818   .        0.00   0.00   0.00  ...   0.00   0.00  0.00

[63285 rows x 429 columns]

In [17]:
y

K562
chrom chromStart chromEnd  strand      
chr10 100006381  100006637 .          0
      100008146  100008402 .          0
      100014418  100014674 .          0
      100020216  100020472 .          0
      100043528  100043784 .          0
...                                 ...
chrY  7520247    7520503   .          0
      7724272    7724528   .          0
      7770029    7770285   .          0
      7796295    7796551   .          0
      8007562    8007818   .          0

[63285 rows x 1 columns]

count the NaNs

In [18]:
print(f"The total number of nan is: {X.isna().sum().sum()}")
print(f"Max nan in cols: {X.isna().sum().max()}")

The total number of nan is: 102
Max nan in cols: 93


In [19]:
imputer = KNNImputer() # default to 5 neighbours
# fit_transform creates an nparray, so it's necessary to convert back to DataFrame
X = pd.DataFrame(
    imputer.fit_transform(X),
    index = X.index,
    columns = X.columns
)

scaling the values

`RobustScaler` scales values in range of 10-90 percentiles.

In [20]:
scaler = RobustScaler()
scaler.fit_transform(X)

array([[ 0.31578947,  0.43902439,  1.03278689, ...,  2.14666667,
         0.08333333, -0.32142857],
       [ 0.01754386,  0.82926829,  0.        , ...,  3.12      ,
        -0.46666667,  0.32142857],
       [ 0.0877193 , -0.95121951, -0.3442623 , ..., -0.21333333,
         0.33333333,  0.16071429],
       ...,
       [-1.26315789, -1.75609756, -1.21311475, ..., -1.02666667,
        -1.28333333, -1.55357143],
       [-1.26315789, -1.75609756, -1.21311475, ..., -1.02666667,
        -1.28333333, -1.55357143],
       [-1.26315789, -1.75609756, -1.21311475, ..., -1.02666667,
        -1.28333333, -1.55357143]])

In [59]:
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
from multiprocessing import cpu_count

@Cache(
    cache_path=[
        "active_enhancers_performance/{function_name}/kept_features_{_hash}.json",
        "active_enhancers_performance/{function_name}/discarded_features_{_hash}.json"
    ],
    args_to_ignore=[
        "X_train", "y_train"
    ]
)
def execute_boruta_feature_selection(
    X_train: pd.DataFrame,
    y_train: np.ndarray,
    holdout_number: int,
    max_iter: int = 100
):
    """Returns tuple with list of kept features and list of discared features.
    
    Parameters
    --------------------------
    X_train: pd.DataFrame,
        The data reserved for the input of the training of the Boruta model.
    y_train: np.ndarray,
        The data reserved for the output of the training of the Boruta model.
    holdout_number: int,
        The current holdout number.
    max_iter: int = 100,
        Number of iterations to run Boruta for.
    """
    # Create the Boruta model
    boruta_selector = BorutaPy(
        # Defining the model that Boruta should use.
        RandomForestClassifier(
            n_jobs=cpu_count(),
            class_weight='balanced_subsample',
            max_depth=5
        ),
        # We leave the number of estimators to be decided by Boruta
        n_estimators='auto',
        verbose=False,
        alpha=0.05, # p_value
        # In practice one would run at least 100-200 times,
        # until all tentative features are exausted.
        max_iter=max_iter, 
        random_state=42,
    )
    # Fit the Boruta model
    boruta_selector.fit(X_train.values, y_train)
    
    # Get the kept features and discarded features
    kept_features = list(X_train.columns[boruta_selector.support_])
    discarded_features = list(X_train.columns[~boruta_selector.support_])
    
    # Filter out the unused featured.
    return kept_features, discarded_features

In [21]:
# method to evaluate the models

def evaluate_model_prediction(
    y_true: np.ndarray,
    y_pred: np.ndarray
) -> Dict[str, float]:
  """Return the evaluation of the given predictions.

  Parameters
  -----------------------------
  y_true: np.ndarray,
    the ground truth labels.
  y_pred: np.ndarray,
    the predicted labels.
  
  Raises
  -----------------------------
  ValueError,
    if the two given array do no have the same shape.

  Returns
  -----------------------------
  Dictionary with the performance metrics.
  """
  y_true = y_true.flatten()
  y_pred = y_pred.flatten()
  if y_true.shape != y_pred.shape:
    raise ValueError(
        "The two arrays do not have the same shape: {} != {}".format(
            y.true.shape, y_pred.shape
        )
    )

  float_metrics = average_precision_score, roc_auc_score
  int_metrics = (accuracy_score, ) # tuple of one elem
  return {
      ** { # `**` unwrap the dict as a linear sequence of key-value pairs
          sanitize_ml_labels(metric.__name__): metric(
              y_true,
              y_pred
          )
          for metric in float_metrics
      },
      ** {
          sanitize_ml_labels(metric.__name__): metric(
              y_true, 
              np.round(y_pred).astype(int)
          )
          for metric in int_metrics
      }
  }

In [60]:
def all_model_evalutation(
    y_train_true: np.ndarray,
    y_train_pred: np.ndarray,
    y_test_true: np.ndarray,
    y_test_pred: np.ndarray,
    model_name: str,
    holdout_number: int,
    use_feature_selection: bool
) -> List[Dict]:

  info = {
      "model_name": model_name,
      "holdout_number": holdout_number,
      "use_feature_selection": use_feature_selection
  }

  return [
      {
        ** evaluate_model_prediction(y_train_true, y_train_pred),
        "run_type": "train",
        ** info
      },
      {
        ** evaluate_model_prediction(y_test_true, y_test_pred),
        "run_type": "test",
        ** info  
      },
  ]

In [62]:
@Cache(
    cache_path="active_enhancers/{function_name}/{_hash}.json",
    args_to_ignore=[
      "X_train", "X_test", "y_train", "y_test"
    ]
)
def train_decision_tree(
    X_train: np.ndarray, 
    X_test: np.ndarray, 
    y_train: np.ndarray,
    y_test: np.ndarray,
    holdout_number: int,
    use_feature_selection: bool
) -> Dict[str, float]:
  tree = DecisionTreeClassifier(
      max_depth = 10,
      min_samples_leaf = 50
  )
  tree.fit(X_train,y_train)
  y_train_pred = tree.predict(X_train)
  y_test_pred = tree.predict(X_test)
  return all_model_evalutation(
      y_train, 
      y_train_pred,
      y_test,
      y_test_pred,
      "Decision tree",
      holdout_number,
      use_feature_selection
  )

In [63]:
from sklearn.ensemble import RandomForestClassifier
from multiprocess import cpu_count

@Cache(
    cache_path="active_enhancers/{function_name}/{_hash}.json",
    args_to_ignore=[
      "X_train", "X_test", "y_train", "y_test"
    ]
)
def train_random_forest(
    X_train: np.ndarray, 
    X_test: np.ndarray, 
    y_train: np.ndarray,
    y_test: np.ndarray,
    holdout_number: int,
    use_feature_selection: bool
) -> Dict[str, float]:
  forest = RandomForestClassifier(
      n_estimators = 600,
      class_weight = "balanced_subsample",
      max_depth = 5,
      min_samples_leaf = 100,
      n_jobs = cpu_count(),
      verbose=True
  )
  forest.fit(X_train,y_train)
  y_train_pred = forest.predict(X_train)
  y_test_pred = forest.predict(X_test)
  return all_model_evalutation(
      y_train, 
      y_train_pred,
      y_test,
      y_test_pred,
      "Random Forest",
      holdout_number,
      use_feature_selection
  )

In [69]:
from tensorflow.keras.layers import Dense, InputLayer
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from extra_keras_metrics import get_minimal_multiclass_metrics

@Cache(
    cache_path="active_enhancers/{function_name}/{_hash}.json",
    args_to_ignore=[
        "X_train", "X_test", "y_train", "y_test"
    ]
)
def train_perceptron(
    X_train: np.ndarray,
    X_test: np.ndarray,
    y_train: np.ndarray,
    y_test: np.ndarray,
    holdout_number: int,
    use_feature_selection: bool
) -> Dict[str, float]:
    """Return performance of a Perceptron.
    
    Parameters
    ----------------------
    X_train: np.ndarray,
        Data reserved for the input during training of the model.
    X_test: np.ndarray,
        Data reserved for the input during  test of the model.
    y_train: np.ndarray,
        Data reserved for the output during  training of the model.
    y_test: np.ndarray,
        Data reserved for the output during  test of the model.
    holdout_number: int,
        Number of the holdout.
    use_feature_selection: bool,
        Whether the model is trained using features that have
        been selected with Boruta or not.
        
    Returns
    ----------------------
    Dictionary with the model perfomance.
    """
    perceptron = Sequential()
    perceptron.add(Dense(
        1,
        activation="sigmoid",
        input_shape=( (X_train.shape[1], ) )
    ))

    perceptron.compile(
        loss="binary_crossentropy",
        optimizer="nadam",
        metrics=get_minimal_multiclass_metrics()
    )
    perceptron.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=1000,
        batch_size=1024,
        verbose=False,
        callbacks=[
            EarlyStopping(
                monitor="val_loss",
                patience=10
            ),
            TqdmCallback(verbose=1)
        ]
    )
    y_train_pred = perceptron.predict(X_train)
    y_test_pred = perceptron.predict(X_test)
    return all_model_evalutation(
        y_train,
        y_train_pred,
        y_test,
        y_test_pred,
        "Perceptron",
        holdout_number,
        use_feature_selection
    )

In [64]:
@Cache(
    cache_path="active_enhancers/{function_name}/{_hash}.json",
    args_to_ignore=[
        "X_train", "X_test", "y_train", "y_test"
    ]
)
def train_ffnn(
    X_train: np.ndarray,
    X_test: np.ndarray,
    y_train: np.ndarray,
    y_test: np.ndarray,
    holdout_number: int,
    use_feature_selection: bool
) -> Dict[str, float]:
    """Return performance of a Feed Forward Neural Network.
    
    Parameters
    ----------------------
    X_train: np.ndarray,
        Data reserved for the input during training of the model.
    X_test: np.ndarray,
        Data reserved for the input during  test of the model.
    y_train: np.ndarray,
        Data reserved for the output during  training of the model.
    y_test: np.ndarray,
        Data reserved for the output during  test of the model.
    holdout_number: int,
        Number of the holdout.
    use_feature_selection: bool,
        Whether the model is trained using features that have
        been selected with Boruta or not.
        
    Returns
    ----------------------
    Dictionary with the model perfomance.
    """
    ffnn = Sequential([
        InputLayer( (X_train.shape[1], ) ),
        Dense(128, activation="relu"),
        Dense(1, activation="sigmoid")
    ])
    ffnn.compile(
        loss="binary_crossentropy",
        optimizer="nadam",
        metrics=get_minimal_multiclass_metrics()
    )
    ffnn.fit(
        X_train,
        y_train,
        validation_data=(X_test, y_test),
        epochs=1000,
        batch_size=1024,
        verbose=False,
        callbacks=[
            EarlyStopping(
                monitor="val_loss",
                patience=10
            ),
            TqdmCallback(verbose=1)
        ]
    )
    y_train_pred = ffnn.predict(X_train)
    y_test_pred = ffnn.predict(X_test)
    return all_model_evalutation(
        y_train,
        y_train_pred,
        y_test,
        y_test_pred,
        "FFNN",
        holdout_number,
        use_feature_selection
    )

In [29]:
number_of_splits = 10

holdouts_generator = StratifiedShuffleSplit(
  n_splits = number_of_splits,
  test_size = 0.2, # 20% of the dataset as test set
)

In [ ]:
all_performance = []

for holdout_number, (train_indices, test_indices) in tqdm(
    # returns a generator over indices of train/test set
    enumerate(holdouts_generator.split(X, y)),
    total=number_of_splits,
    desc="Computing holdouts"
):
  for use_feature_selection in tqdm(
      (True, False),
      desc="Running feature selection",
      leave=False
  ):

    X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]

    # do it once for all the models
    if use_feature_selection:
      kept_features, discarded_features = execute_boruta_feature_selection(
          X_train,
          y_train.values.ravel(),
          holdout_number
      )
      # We filter the DataFrames columns using the features that
      # Boruta has decided we should keep
      X_train = X_train[kept_features]
      X_test = X_test[kept_features]
    
    for train_model in tqdm(
        (train_decision_tree, train_perceptron, train_ffnn, train_random_forest),
        desc="Training model",
        leave=False
    ):
      performance = train_model(
          X_train.values,
          X_test.values,
          y_train.values,
          y_test.values,
          holdout_number
      )
      all_performance += performance
    break

all_performance = pd.DataFrame(all_performance)

In [ ]:
# !rm -rf active_enhancers/train_perceptron/

In [ ]:
all_performance

# wilcoxon test

use it when you need to evaluate which model
outperform another model, and also to understand if it is important to perform feature selection or not.

In [ ]:
# from scipy.stats import wilcoxon
# 
# # Here we will be doing a statistical test.
# models = df[
#     (df.run_type == "test")
# ]
# 
# ffnn_scores = models[models.model=="Perceptron"]
# mlp_scores = models[models.model=="DecisionTreeClassifier"]
# 
# alpha = 0.01
# 
# for metric in ffnn_scores.columns[-4:]:
#     print(metric)
#     a,  b = ffnn_scores[metric], mlp_scores[metric]
#     stats, p_value = wilcoxon(a, b)
#     if p_value > alpha:
#         print(p_value, "The two models performance are statistically identical.")
#     else:
#         print(p_value, "The two models performance are different")
#         if a.mean() > b.mean():
#             print("The first model is better")
#         else:
#             print("The second model is better")